In [16]:
import os
import sys
sys.path.append('../')
sys.path.append('../datasets')
sys.path.append('../models')
import torch
import torchvision
import torch.backends.cudnn as cudnn
import random
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from importlib import reload
from datasets.three_dim_shapes import ThreeDimShapesDataset
from datasets.small_norb import SmallNORBDataset
from datasets.seq_mnist import SequentialMNIST
import models.seqae as seqae
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from datasets import so3_data as so3d
from einops import rearrange

import csv
import ast



from tqdm import tqdm

if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
else:
    device = torch.device('cpu')
    gpu_index = -1

    
    
from utils import yaml_utils as yu
import yaml
result_dir = '/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result'





In [17]:
ls /mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result/*so3*

/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result/20220511_so3_exp_0:
C_bd0/  C_bd0id1/  C_bd0id2/  config.yml  log

/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result/20220611_so3:
config.yml                    snapshot_model_iter_20000
log                           snapshot_model_iter_25000
snapshot_manager_iter_100000  snapshot_model_iter_30000
snapshot_model_iter_10000     snapshot_model_iter_40000
snapshot_model_iter_100000    snapshot_model_iter_50000
snapshot_model_iter_125000    snapshot_model_iter_75000

/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result/20220611_so3_so3run:
config.yml                    snapshot_model_iter_175000
log                           snapshot_model_iter_200000
snapshot_manager_iter_200000  snapshot_model_iter_25000
snapshot_model_iter_100000    snapshot_model_iter_50000
snapshot_model_iter_125000    snapshot_model_iter_75000
snapshot_model_iter_150000

/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result/20220611_so3_so3run_rcst:
config.yml   

In [18]:
targdir = '20220628_so3_linear_net_0/T8'
exp_result = os.path.join(result_dir, targdir)
print(os.listdir(exp_result))
targ_config_path = os.path.join(exp_result, 'config.yml')


with open(targ_config_path, 'r') as f:
    config = yaml.safe_load(f)
    
    
def load_model(model, log_dir, iters):
    print('snapshot_model_iter_{}'.format(iters))
    model.load_state_dict(torch.load(os.path.join(
        log_dir, 'snapshot_model_iter_{}'.format(iters))))





['log', 'snapshot_manager_iter_10000', 'snapshot_model_iter_10000', 'config.yml']


In [19]:
data = yu.load_component(config['train_data'])
train_loader = DataLoader(data, 
                          batch_size=config['batchsize'],
                          shuffle=True,
                          num_workers=config['num_workers'])
model = yu.load_component(config['model'])
load_model(model, exp_result, 10000)

snapshot_model_iter_10000


/usr/local/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [20]:
angle_dats = [] 
axis_dats = []
for k in range(len(data.labels)):
    angledat = [data.labels[k]['blocks'][j]['angle'] for j in range(len(data.labels[0]['blocks']))]
    axisdat = torch.stack([data.labels[k]['blocks'][j]['axis'] for j in range(len(data.labels[0]['blocks']))])

    angle_dats.append(angledat)
    axis_dats.append(axisdat)
    
angle_dats = torch.tensor(angle_dats)
axis_dats = torch.stack(axis_dats)
print(angle_dats.shape)
print(axis_dats.shape)
label_dats = {'angle':angle_dats, 'axis':axis_dats}



torch.Size([10000, 2])
torch.Size([10000, 2, 3])


In [21]:
Ms_list = []
device = 0
model = model.to(device)
for idx, images in enumerate(train_loader):
    images = images.to(device)
    Ms = model.get_M(images).detach().to('cpu')
    Ms_list.append(Ms)
    
Ms_all = torch.cat(Ms_list, axis=0)

In [30]:
scores = [] 
for idx, dat in enumerate(train_loader):
    dat = rearrange(dat, 'b t a -> (b t) a' ).to(device)
    d0hat =  model.dec(model.enc(dat)).detach().to(device)
    
    scores.append(seqae.r2_score(d0hat.to('cpu'), dat.to('cpu')))
    
scores = torch.tensor(scores)
print(torch.mean(scores))

tensor(0.9998, dtype=torch.float64)
